In [1]:
# Import the dependencies 
import pandas as pd
import gmaps
import requests

# Import the API key
from config import g_key

In [3]:
# Store the CSV created into a Dataframe 
city_data_df = pd.read_csv("weather_data/cities.csv")

# Check to see the data columns are integers/ floating decimals
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
## Create a Maximum Temp. Heatmap 

In [5]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level= 1.5)

# Assign the heatmap variable.
# remove negative temperatures bc can't be graphed 
heat_layer = gmaps.heatmap_layer(locations, weights= [max(temp, 0) for temp in max_temp], 
                                 dissipating= False, max_intensity= 300, point_radius= 4)

#Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig 

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

fig= gmaps.figure(center=(30.0, 31.0), zoom_level= 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating= False, max_intensity= 300, point_radius= 4)
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]

fig= gmaps.figure(center=(30.0, 31.0), zoom_level= 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights= clouds, 
                                 dissipating= False, max_intensity= 300, point_radius= 4)
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent wind speed 
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]

fig= gmaps.figure(center=(30.0, 31.0), zoom_level= 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights= wind, 
                                 dissipating= False, max_intensity= 300, point_radius= 4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?60
What is the maximum temperature you would like for your trip?75


In [11]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Lepe,ES,2022-09-25 00:10:44,37.2548,-7.2043,68.11,58,59,14.92
11,11,Katsuura,JP,2022-09-25 00:08:23,35.1333,140.3000,73.78,67,88,21.52
14,14,Port Alfred,ZA,2022-09-25 00:10:51,-33.5906,26.8910,65.17,92,100,5.01
18,18,Lagoa,PT,2022-09-25 00:10:53,39.0500,-27.9833,71.71,80,20,21.50
21,21,Chimbote,PE,2022-09-25 00:10:56,-9.0853,-78.5783,62.01,84,83,10.18


In [13]:
preferred_cities_df.count()

City_ID       184
City          184
Country       181
Date          184
Lat           184
Lng           184
Max Temp      184
Humidity      184
Cloudiness    184
Wind Speed    184
dtype: int64

In [18]:
clean = preferred_cities_df.dropna(axis= "index", how= "any")

clean.count()

City_ID       181
City          181
Country       181
Date          181
Lat           181
Lng           181
Max Temp      181
Humidity      181
Cloudiness    181
Wind Speed    181
dtype: int64

In [19]:
# 6.5.4 
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates

hotel_df = clean[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Lepe,ES,68.11,37.2548,-7.2043,
11,Katsuura,JP,73.78,35.1333,140.3000,
14,Port Alfred,ZA,65.17,-33.5906,26.8910,
18,Lagoa,PT,71.71,39.0500,-27.9833,
21,Chimbote,PE,62.01,-9.0853,-78.5783,


In [20]:
# Set the parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [21]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Lepe,ES,68.11,37.2548,-7.2043,Hotel La Barca
11,Katsuura,JP,73.78,35.1333,140.3000,Katsuura Hotel Mikazuki
14,Port Alfred,ZA,65.17,-33.5906,26.8910,The Halyards Hotel
18,Lagoa,PT,71.71,39.0500,-27.9833,Casa Das Faias
21,Chimbote,PE,62.01,-9.0853,-78.5783,Hotel Briliá


In [25]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)

# add marker layer for location (lat and lng)
marker_layer = gmaps.marker_layer(locations)

# add both layers 
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))